In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Load_Data").getOrCreate()

In [ ]:
df_with_abstract = spark.read.parquet("/content/drive/MyDrive/df_with_abstract.parquet")

In [ ]:
from pyspark.sql.functions import col


In [ ]:
new_pmid_list  = df_with_abstract.select(col("MedlineCitation.PMID.#text")).rdd.flatMap(lambda x: x).collect()
new_pmid_list  = [int(pmid) for pmid in new_pmid_list ]

In [ ]:
new_pmid_list

[28343685,
 28343686,
 28343687,
 28343691,
 28343692,
 28343693,
 28343694,
 28343695,
 28343696,
 28343697,
 28343698,
 28343699,
 28343700,
 28343701,
 28343703,
 28343707,
 28343709,
 28343710,
 28343712,
 28343711,
 28343714,
 28343715,
 28343716,
 28343713,
 28343718,
 28343719,
 28343720,
 28343723,
 28343725,
 28343726,
 28343727,
 28343728,
 28343729,
 28343731,
 28343732,
 28343734,
 28343733,
 28343736,
 28343737,
 28343738,
 28343741,
 28343742,
 28343739,
 28343744,
 28343745,
 28343740,
 28343746,
 28343747,
 28343748,
 28343749,
 28343750,
 28343751,
 28343752,
 28343753,
 28343755,
 28343756,
 28343757,
 28343759,
 28343760,
 28343761,
 28343758,
 28343762,
 28343763,
 28343764,
 28343765,
 28343766,
 28343767,
 28343769,
 28343770,
 28343768,
 28343771,
 28343773,
 28343772,
 28343774,
 28343775,
 28343776,
 28343777,
 28343779,
 28343780,
 28343781,
 28343782,
 28343783,
 28343784,
 28343785,
 28343786,
 28343787,
 28343792,
 28343791,
 28343793,
 28343794,
 28343795,

In [ ]:
total_pmids = len(pmid_list)

NameError: name 'pmid_list' is not defined

In [ ]:
total_pmids

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd


batch_size = 100

def fetch_citations_batch(pmids):
    """
    Fetch citations for a batch of PMIDs using the PubMed E-utilities API.
    Returns a dict mapping each PMID to a list of citing article PMIDs.
    """
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi"
    pmids_str = [str(pmid) for pmid in pmids]
    params = {
        "dbfrom": "pubmed",
        "linkname": "pubmed_pubmed_citedin",
        "id": ",".join(pmids_str)
    }
    citing_dict = {}
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching batch {pmids[:5]}...: {e}")
        for pmid in pmids:
            citing_dict[pmid] = []
        return citing_dict

    root = ET.fromstring(response.text)
    for linkset in root.findall('LinkSet'):
        pmid_elem = linkset.find('./IdList/Id')
        if pmid_elem is None:
            continue
        pmid = pmid_elem.text
        citing_ids = []
        linkset_db = linkset.find('LinkSetDb')
        if linkset_db is not None:
            for link in linkset_db.findall('./Link'):
                cid = link.find('Id')
                if cid is not None:
                    citing_ids.append(cid.text)
        citing_dict[pmid] = citing_ids
    return citing_dict

In [ ]:
existing_pmids_df = pd.read_csv("/content/drive/MyDrive/PubMedData/pmid_citations_901.csv")
existing_pmids = existing_pmids_df['PMID'].tolist()

In [ ]:
existing_pmids

[28343694,
 28343695,
 28343697,
 28343701,
 28343720,
 28343731,
 28343748,
 28343759,
 28343761,
 28343772,
 28343781,
 28343791,
 28343808,
 28343809,
 28343820,
 28343818,
 28343835,
 28343854,
 28343865,
 28343867,
 28343868,
 28343885,
 28343904,
 28343909,
 28343940,
 28343960,
 28343962,
 28343966,
 28343970,
 28343983,
 28343984,
 28343996,
 28343999,
 28344000,
 28344001,
 28344002,
 28344003,
 28344004,
 28344005,
 28344017,
 28344019,
 28344040,
 28344039,
 28344061,
 28344067,
 28344068,
 28344070,
 28344075,
 28344076,
 28344080,
 28344082,
 28344110,
 28344118,
 28344120,
 28344124,
 28344128,
 28344129,
 28344136,
 28344151,
 28344172,
 28344192,
 28344194,
 28344196,
 28344195,
 28344197,
 28344215,
 28344214,
 28344218,
 28344216,
 28344219,
 28344221,
 28344217,
 28344220,
 28344222,
 28344223,
 28344225,
 28344224,
 28344226,
 28344227,
 28344231,
 28344230,
 28344229,
 28344233,
 28344232,
 28344234,
 28344235,
 28344237,
 28344238,
 28344236,
 28344239,
 28344240,

In [ ]:
pmids_to_process = [pmid for pmid in new_pmid_list if pmid not in existing_pmids]

In [ ]:
len(pmids_to_process)

12899

In [ ]:
!pip install requests pandas xml.etree.ElementTree
import requests
import xml.etree.ElementTree as ET
import pandas as pd

ERROR: Could not find a version that satisfies the requirement xml.etree.ElementTree (from versions: none)
ERROR: No matching distribution found for xml.etree.ElementTree


In [ ]:
# import time
# import pandas as pd

# results = []
# batch_size = 100
# all_citations = {}

# for i, pmid in enumerate(pmids_to_process):
#     citations = fetch_citations_batch([pmid])
#     all_citations[pmid] = citations[str(pmid)]

#     df_citations = pd.DataFrame(list(all_citations.items()), columns=['PMID', 'Citations'])
#     df_citations.to_csv("remaining_pmid_citations.csv", index=False)
#     print(f"Saved {len(df_citations)} records to remaining_pmid_citations.csv after processing PMID: {pmid}")

#     if (i + 1) % 1 == 0:
#         time.sleep(3)

Streaming output truncated to the last 5000 lines.
Saved 656 records to remaining_pmid_citations.csv after processing PMID: 28345518
Saved 657 records to remaining_pmid_citations.csv after processing PMID: 28345516
Saved 658 records to remaining_pmid_citations.csv after processing PMID: 28345525
Saved 659 records to remaining_pmid_citations.csv after processing PMID: 28345526
Saved 660 records to remaining_pmid_citations.csv after processing PMID: 28345527
Saved 661 records to remaining_pmid_citations.csv after processing PMID: 28345528
Saved 662 records to remaining_pmid_citations.csv after processing PMID: 28345520
Saved 663 records to remaining_pmid_citations.csv after processing PMID: 28345529
Saved 664 records to remaining_pmid_citations.csv after processing PMID: 28345530
Saved 665 records to remaining_pmid_citations.csv after processing PMID: 28345531
Saved 666 records to remaining_pmid_citations.csv after processing PMID: 28345532
Saved 667 records to remaining_pmid_citations.c

KeyError: '28359763'

In [ ]:
# import time
# import pandas as pd



# try:
#     remaining_citations_df = pd.read_csv("remaining_pmid_citations.csv")
#     processed_pmids = remaining_citations_df['PMID'].tolist()
#     all_citations = dict(zip(remaining_citations_df['PMID'], remaining_citations_df['Citations']))
# except FileNotFoundError:
#     remaining_citations_df = pd.DataFrame(columns=['PMID', 'Citations'])
#     processed_pmids = []
#     all_citations = {}

# # Filter new PMIDs and exclude already processed ones
# pmids_to_process = [pmid for pmid in new_pmid_list if pmid not in existing_pmids and pmid not in processed_pmids]


# results = []
# batch_size = 100

# for i, pmid in enumerate(pmids_to_process):
#     try:
#         citations = fetch_citations_batch([pmid])
#         pmid_str = str(pmid)
#         if pmid_str in citations:
#             all_citations[pmid] = citations[pmid_str]
#         else:
#             print(f"Warning: No citations found for PMID {pmid}")
#             all_citations[pmid] = []

#         # Append new citations to the DataFrame
#         new_row = pd.DataFrame([[pmid, all_citations[pmid]]], columns=['PMID', 'Citations'])
#         remaining_citations_df = pd.concat([remaining_citations_df, new_row], ignore_index=True)

#         remaining_citations_df.to_csv("remaining_pmid_citations.csv", index=False)
#         print(f"Saved {len(remaining_citations_df)} records to remaining_pmid_citations.csv after processing PMID: {pmid}")

#         if (i + 1) % 1 == 0:  # Adjust sleep frequency if needed
#             time.sleep(3)

#     except (KeyError, requests.exceptions.RequestException) as e:
#         print(f"Error processing PMID {pmid}: {e}")
#         # Optionally, log the error:
#         # with open("error_log.txt", "a") as error_log:
#         #     error_log.write(f"Error processing PMID {pmid}: {e}\n")


Streaming output truncated to the last 5000 lines.
Saved 6700 records to remaining_pmid_citations.csv after processing PMID: 28362783
Saved 6701 records to remaining_pmid_citations.csv after processing PMID: 28362785
Saved 6702 records to remaining_pmid_citations.csv after processing PMID: 28362786
Saved 6703 records to remaining_pmid_citations.csv after processing PMID: 28362787
Saved 6704 records to remaining_pmid_citations.csv after processing PMID: 28362788
Saved 6705 records to remaining_pmid_citations.csv after processing PMID: 28362791
Saved 6706 records to remaining_pmid_citations.csv after processing PMID: 28362790
Saved 6707 records to remaining_pmid_citations.csv after processing PMID: 28362792
Saved 6708 records to remaining_pmid_citations.csv after processing PMID: 28362793
Saved 6709 records to remaining_pmid_citations.csv after processing PMID: 28362794
Saved 6710 records to remaining_pmid_citations.csv after processing PMID: 28362795
Saved 6711 records to remaining_pmid

In [ ]:
import time
import pandas as pd



try:
    remaining_citations_df = pd.read_csv("/content/drive/MyDrive/df_with_abstract/remaining_pmid_citations.csv")
    processed_pmids = remaining_citations_df['PMID'].tolist()
except FileNotFoundError:
    remaining_citations_df = pd.DataFrame(columns=['PMID', 'Citations'])
    processed_pmids = []

start_pmid = 28368885
pmids_to_process = [pmid for pmid in new_pmid_list if pmid >= start_pmid and pmid not in existing_pmids and pmid not in processed_pmids]


results = []
batch_size = 100

for i, pmid in enumerate(pmids_to_process):
    try:
        citations = fetch_citations_batch([pmid])
        pmid_str = str(pmid)
        if pmid_str in citations:
            all_citations[pmid] = citations[pmid_str]
        else:
            print(f"Warning: No citations found for PMID {pmid}")
            all_citations[pmid] = []

        # Append new citations to the DataFrame
        new_row = pd.DataFrame([[pmid, all_citations[pmid]]], columns=['PMID', 'Citations'])
        remaining_citations_df = pd.concat([remaining_citations_df, new_row], ignore_index=True)

        remaining_citations_df.to_csv("remaining_pmid_citations.csv", index=False)
        print(f"Saved {len(remaining_citations_df)} records to remaining_pmid_citations.csv after processing PMID: {pmid}")

        if (i + 1) % 1 == 0:  # Adjust sleep frequency if needed
            time.sleep(3)

    except (KeyError, requests.exceptions.RequestException) as e:
        print(f"Error processing PMID {pmid}: {e}")
        # Optionally, log the error:
        # with open("error_log.txt", "a") as error_log:
        #     error_log.write(f"Error processing PMID {pmid}: {e}\n")

Saved 1 records to remaining_pmid_citations.csv after processing PMID: 28368886
Saved 2 records to remaining_pmid_citations.csv after processing PMID: 28368889
Saved 3 records to remaining_pmid_citations.csv after processing PMID: 28368891
Saved 4 records to remaining_pmid_citations.csv after processing PMID: 28368888
Saved 5 records to remaining_pmid_citations.csv after processing PMID: 28368892
Saved 6 records to remaining_pmid_citations.csv after processing PMID: 28368887
Saved 7 records to remaining_pmid_citations.csv after processing PMID: 28368885
Saved 8 records to remaining_pmid_citations.csv after processing PMID: 28368894
Saved 9 records to remaining_pmid_citations.csv after processing PMID: 28368895
Saved 10 records to remaining_pmid_citations.csv after processing PMID: 28368893
Saved 11 records to remaining_pmid_citations.csv after processing PMID: 28368890
Saved 12 records to remaining_pmid_citations.csv after processing PMID: 28368898
Saved 13 records to remaining_pmid_ci

In [ ]:
remaining_citations_df.to_csv("/content/drive/My Drive/remaining_pmid_citations.csv", index=False)
print("Saved permanently in Google Drive: remaining_pmid_citations.csv")


Saved permanently in Google Drive: remaining_pmid_citations.csv
